In [1]:
import pandas as pd

Reading in a small sample. Crimes are sorted by ID in the raw data so it isn't from one specific time period.

In [22]:
crimeDF = pd.read_csv("../data/raw/Crimes_-_2001_to_present.csv", nrows = 8000, skiprows = range(1,200000), header = 0)

In [23]:
crimeDF.sort_values("Date")

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
3177,9806042,HX454957,01/01/2001 09:00:00 AM,038XX N HAMLIN AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,39.0,16.0,11,NaN,NaN,2001,08/31/2015 03:43:09 PM,NaN,NaN,NaN
2891,9755080,HX403536,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2870,9754975,HX403528,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2876,9754865,HX403519,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2877,9755112,HX403539,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2878,9754894,HX403520,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2881,9754946,HX403521,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2883,9754953,HX403523,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2884,9754963,HX403524,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
2886,9754968,HX403526,01/01/2001 12:00:00 AM,031XX W DOUGLAS BLVD,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,CHURCH/SYNAGOGUE/PLACE OF WORSHIP,False,False,...,24.0,29.0,17,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN


In [26]:
crimeDF[crimeDF["Latitude"].notnull()]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
280,8929938,HV601471,12/13/2012 09:18:00 PM,016XX N HARLEM AVE,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,True,False,...,36.0,25.0,06,1127853.0,1909983.0,2012,02/04/2016 06:33:39 AM,41.909307,-87.805767,"(41.909307068, -87.80576669)"
1028,8792068,HV465694,09/06/2012 11:00:00 PM,066XX W ALBION AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,...,41.0,10.0,06,1131021.0,1943405.0,2012,02/04/2016 06:33:39 AM,42.000967,-87.793355,"(42.000966584, -87.793355012)"
1031,8793163,HV467220,09/08/2012 12:00:00 PM,023XX E 70TH PL,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,5.0,43.0,08A,1193173.0,1858670.0,2012,02/04/2016 06:33:39 AM,41.767149,-87.567493,"(41.767148983, -87.567493189)"
1039,9308854,HW453686,09/16/2013 04:56:00 AM,048XX S WOOD ST,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,20.0,61.0,08A,1165165.0,1872576.0,2013,02/04/2016 06:33:39 AM,41.805947,-87.669760,"(41.805947267, -87.669760387)"
1041,8799283,HV473138,09/11/2012 09:30:00 PM,009XX W DAKIN ST,0810,THEFT,OVER $500,STREET,False,False,...,46.0,6.0,06,1169309.0,1926418.0,2012,02/04/2016 06:33:39 AM,41.953605,-87.652996,"(41.953604821, -87.652996353)"
1042,8802469,HV475762,09/14/2012 04:47:00 PM,036XX S STATE ST,0810,THEFT,OVER $500,LIBRARY,False,False,...,3.0,35.0,06,1176894.0,1880780.0,2012,02/04/2016 06:33:39 AM,41.828203,-87.626495,"(41.828203205, -87.626495474)"
1051,8857760,HV531298,10/23/2012 04:28:00 PM,017XX W HOWARD ST,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,...,49.0,1.0,06,1163126.0,1950347.0,2012,02/04/2016 06:33:39 AM,42.019399,-87.675049,"(42.019399237, -87.675049485)"
1052,8857949,HV531603,10/23/2012 07:45:00 PM,002XX W 45TH PL,0560,ASSAULT,SIMPLE,RESIDENCE,False,True,...,3.0,37.0,08A,1175263.0,1874770.0,2012,02/04/2016 06:33:39 AM,41.811748,-87.632659,"(41.811747899, -87.632659218)"
1053,9455783,HX108475,01/09/2014 04:10:00 PM,022XX W 24TH ST,0460,BATTERY,SIMPLE,STREET,False,False,...,25.0,31.0,08B,1161598.0,1888036.0,2014,02/04/2016 06:33:39 AM,41.848446,-87.682413,"(41.848446094, -87.682413255)"
1054,8859866,HV533611,10/25/2012 06:30:00 AM,031XX W OHIO ST,0460,BATTERY,SIMPLE,RESIDENCE,False,True,...,27.0,23.0,08B,1155162.0,1903825.0,2012,02/04/2016 06:33:39 AM,41.891904,-87.705610,"(41.891904216, -87.705610152)"


### Since the coordinates are frequently not available, we will have to use the ward to determine approximate location, and map it to Zip Code using the following data

In [27]:
wardDF = pd.read_csv("../data/raw/Ward_Offices.csv")

In [29]:
wardDF.head()

,WARD,ALDERMAN,ADDRESS,CITY,STATE,ZIPCODE,WARD PHONE,WARD FAX,EMAIL,WEBSITE,LOCATION,CITY HALL ADDRESS,CITY HALL CITY,CITY HALL STATE,CITY HALL ZIPCODE,CITY HALL PHONE
0,1,"Proco ""Joe"" Moreno",2058 North Western Avenue,Chicago,IL,60647,(773) 278-0101,(773) 278-2541,ward01@cityofchicago.org,http://www.ward1.org,"(41.9192937068, -87.6876139388)","121 North LaSalle Street, Room 300",Chicago,IL,60602,(312) 744-3063
1,2,Brian Hopkins,1400 North Ashland Avenue,Chicago,IL,60622,(312)643-2299,NaN,Ward02@cityofchicago.org,http://www.brianhopkinsforchicago.com/,"(41.9070344812, -87.6677379022)","121 North LaSalle Street, Room 200",Chicago,IL,60602,(312) 744-6834
2,3,Pat Dowell,5046 South State Street,Chicago,IL,60609,(773) 373-9273,(773) 373-6852,Pat.Dowell@cityofchicago.org,http://www.dowellfor3rdward.com/,"(41.8023557628, -87.626046431)","121 North LaSalle Street, Room 200",Chicago,IL,60602,(312) 744-8734
3,4,William D. Burns,"435 East 35th Street, 1st Floor",Chicago,IL,60616,(773) 536-8103,(773) 536-7296,ward04@cityofchicago.org,http://www.aldwillburns.com,"(41.8310649884, -87.6155396214)","121 North LaSalle Street, Room 200",Chicago,IL,60602,(312) 744-2690
4,5,Leslie A. Hairston,2325 East 71st Street,Chicago,IL,60649,(773) 324-5555,(773) 324-1585,lhairston@cityofchicago.org,http://www.leslieahairston.com/,"(41.7660318787, -87.5678811948)","121 North LaSalle Street, Room 300",Chicago,IL,60602,(312) 744-6832
